In [4]:
#Task 1
import random

def generate_moves(state):
    return [state * 2 + i for i in range(3)]

def evaluate_state(state):
    return random.randint(1, 100)

def beam_search_game(start_state, beam_width=2, depth_limit=2):
    beam = [(0, [start_state])]

    for _ in range(depth_limit):
        candidates = []
        for eval_score, path in beam:
            current_state = path[-1]
            next_states = generate_moves(current_state)
            for next_state in next_states:
                score = evaluate_state(next_state)
                candidates.append((score, path + [next_state]))

        candidates.sort(key=lambda x: -x[0])
        beam = candidates[:beam_width]

    best_score, best_path = beam[0]
    return best_path, best_score


start = 1
beam_width = 3
depth = 3

path, score = beam_search_game(start_state=start, beam_width=beam_width, depth_limit=depth)
print("Best Move Sequence:", path)
print("Evaluation Score:", score)


Best Move Sequence: [1, 2, 6, 13]
Evaluation Score: 94


In [24]:
#Task 2
import random
import math

def calculate_total_distance(route, points):
    distance = 0
    for i in range(len(route) - 1):
        x1, y1 = points[route[i]]
        x2, y2 = points[route[i + 1]]
        distance += math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    x1, y1 = points[route[-1]]
    x2, y2 = points[route[0]]
    distance += math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return distance

def get_neighbors(route):
    neighbors = []
    n = len(route)
    for i in range(n):
        for j in range(i + 1, n):
            neighbor = list(route)
            neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
            neighbors.append(neighbor)
    return neighbors

def hill_climbing_tsp(points):
    n = len(points)
    current_route = list(range(n))
    random.shuffle(current_route)
    current_distance = calculate_total_distance(current_route, points)

    while True:
        neighbors = get_neighbors(current_route)
        next_route = None
        next_distance = current_distance

        for neighbor in neighbors:
            distance = calculate_total_distance(neighbor, points)
            if distance < next_distance:
                next_route = neighbor
                next_distance = distance

        if next_distance >= current_distance:
            break
        current_route = next_route
        current_distance = next_distance

    return current_route, current_distance

points = [(0, 0), (1, 3), (4, 3), (6, 1), (3, 0)]

route, distance = hill_climbing_tsp(points)

print("Optimized Route (by index):", route)
print("Total Distance Covered:", distance)


Optimized Route (by index): [2, 3, 4, 0, 1]
Total Distance Covered: 15.15298244508295


In [28]:
#Task 3
import random
import math

def calculate_distance(route, cities):
    distance = 0
    for i in range(len(route)):
        x1, y1 = cities[route[i]]
        x2, y2 = cities[route[(i + 1) % len(route)]]
        distance += math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return distance

def generate_population(cities, size):
    population = []
    for _ in range(size):
        route = list(range(len(cities)))
        random.shuffle(route)
        population.append(route)
    return population

def select_parents(population, cities):
    a = random.choice(population)
    b = random.choice(population)
    return a if calculate_distance(a, cities) < calculate_distance(b, cities) else b

def crossover(parent1, parent2):
    size = len(parent1)
    start = random.randint(0, size - 2)
    end = random.randint(start + 1, size - 1)
    child = [None] * size
    child[start:end + 1] = parent1[start:end + 1]

    fill_pos = 0
    for city in parent2:
        if city not in child:
            while child[fill_pos] is not None:
                fill_pos += 1
            child[fill_pos] = city
    return child

def mutate(route, mutation_rate=0.1):
    for i in range(len(route)):
        if random.random() < mutation_rate:
            j = random.randint(0, len(route) - 1)
            route[i], route[j] = route[j], route[i]
    return route

def genetic_algorithm(cities, population_size=100, generations=500):
    population = generate_population(cities, population_size)

    for gen in range(generations):
        new_population = []
        for _ in range(population_size):
            parent1 = select_parents(population, cities)
            parent2 = select_parents(population, cities)
            child = crossover(parent1, parent2)
            child = mutate(child)
            new_population.append(child)
        population = new_population

    best_route = min(population, key=lambda r: calculate_distance(r, cities))
    best_distance = calculate_distance(best_route, cities)
    return best_route, best_distance

cities = [
    (0, 0), (2, 3), (5, 2), (6, 6), (8, 3),
    (7, 0), (3, 7), (1, 5), (4, 4), (6, 2)
]

best_route, best_distance = genetic_algorithm(cities)

print("Best Route (by city index):", best_route)
print("Total Distance:", best_distance)


Best Route (by city index): [2, 3, 6, 7, 1, 0, 5, 4, 8, 9]
Total Distance: 34.06924007402824
